In [ ]:
import datetime
import json
import os
import pathlib

In [ ]:
# Outputs are stored in `directory_outputs` / <output number> / `subdirectory_output`
epsilon = 7
directory_outputs = pathlib.PurePath('..', 'outputs', 'esnet_windowed')
subdirectory_output = pathlib.PurePath(f'{epsilon}_0.0002_50_50')
directories_outputs = list(sorted([
    (
        float(directory_output),
        directory_outputs / directory_output / subdirectory_output,
    )
    for i, directory_output in enumerate(sorted(os.listdir(directory_outputs)))
    if os.path.isdir(directory_outputs / directory_output)
]))

In [ ]:
# Grab the data from the files
outputs = []
times = []
for t, directory_output in directories_outputs:
    with open(directory_output / 'output.json', 'r') as file_output:
        outputs.append(json.load(file_output))
        times.append(t)

In [ ]:
# Determine a consistent set of cluster representatives, their
# coordinates, and their associated data
node_labels_to_representatives = {}
node_representatives_to_labels = {}
node_labels_to_coordinates = {}
node_labels_to_data = {}
node_data_keys = set()
for output in outputs:
    network = output['network']
    graph_data, vertex_data, edge_data = network

    # Cases depending on whether clustering was used
    if 'elements' in vertex_data:
        for index, (node_label, elements, coordinates) in enumerate(zip(graph_data['labels'], vertex_data['elements'], graph_data['coordinates'])):
            node_representative = min(elements)
            for element in elements:
                node_labels_to_representatives[element] = node_representative
            node_labels_to_coordinates[node_representative] = coordinates
            node_labels_to_data[node_representative] = {
                key: value[index]
                for key, value in vertex_data.items()
            }
    else:
        for index, (node_label, coordinates) in enumerate(zip(graph_data['labels'], graph_data['coordinates'])):
            node_labels_to_representatives[node_label] = node_label
            node_labels_to_coordinates[node_label] = coordinates
            node_labels_to_data[node_label] = {
                key: value[index]
                for key, value in vertex_data.items()
            }

    for key in vertex_data.keys():
        node_data_keys.add(key)

node_indices_to_labels = list(sorted(node_labels_to_coordinates))
node_labels_to_indices = {label: index for index, label in enumerate(node_indices_to_labels)}
for node_label, node_representative in node_labels_to_representatives.items():
    node_labels_to_indices[node_label] = node_labels_to_indices[node_representative]
for node_label, node_representative in node_labels_to_representatives.items():
    if node_representative not in node_representatives_to_labels:
        node_representatives_to_labels[node_representative] = [node_label]
    else:
        node_representatives_to_labels[node_representative].append(node_label)

# Combine the computed data into an appropriate format
node_labels = node_indices_to_labels
node_coordinates = [node_labels_to_coordinates[label] for label in node_labels]
node_data = {
    key: [
        node_labels_to_data[label][key] if key in node_labels_to_data[label] else None
        for label in node_labels
    ]
    for key in node_data_keys
}

# Relabel the networks
for output in outputs:
    network = output['network']
    graph_data, vertex_data, edge_data = network

    graph_data_new = {
        'coordinates': node_coordinates,
        'edges': [
            (node_labels_to_indices[source_label], node_labels_to_indices[target_label])
            for source_index, target_index in graph_data['edges']
            for source_label in (graph_data['labels'][source_index],)
            for target_label in (graph_data['labels'][target_index],)
        ],
        'labels': node_indices_to_labels,
        'bounding_box': graph_data['bounding_box'] if 'bounding_box' in graph_data else None
    }
    vertex_data_new = node_data
    edge_data_new = edge_data

    output['network'] = (graph_data_new, vertex_data_new, edge_data_new)

In [ ]:
networks = [
    output['network']
    for output in outputs
]

all_edges = set()
for network in networks:
    graph_data, _, _ = network
    labels = graph_data['labels']
    for edge in graph_data['edges']:
        edge = [labels[edge[0]], labels[edge[1]]]
        edge = (min(edge), max(edge))
        all_edges.add(edge)
all_edges = list(sorted(all_edges))

In [ ]:
networks = [
    output['network']
    for output in outputs
]

for t, network in zip(times, networks):
    graph_data, _, _ = network
    labels = graph_data['labels']
    present_edges = set([
        (min(labeled_edge), max(labeled_edge))
        for edge in graph_data['edges']
        for labeled_edge in ((labels[edge[0]], labels[edge[1]]),)
    ])
    outages = [
        edge
        for edge in all_edges
        if edge not in present_edges
    ]

    if True:
        print(datetime.datetime.fromtimestamp(t / 1000).strftime('%y-%m-%d %H:%M'))
        for outage in outages:
            print(f'\t{outage[0]} -> {outage[1]}')